In [27]:
import nltk
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

with open('Base/words-utf-8.txt', 'r', encoding='utf-8') as file:
    artigos = file.read()

# Separa as pontuações das palavras
def separa_palavras(lista_artigos):
    lista_palavras = []
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras

lista_tokens = tknzr.tokenize(artigos)
lista_palavras = separa_palavras(lista_tokens)

def normalizacao(lista_palavras):
    lista_normalizada = []
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

lista_normalizada = normalizacao(lista_palavras)

# Inserindo as letras esquecidas na hora de digitar a palavra
def insere_letras(fatias) :
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

#Em caso de digitar caracteres a mais na palavra
def deletando_caracteres(fatias):
    novas_palavras = []
    for E, D in fatias:
        novas_palavras.append(E + D[1:])
    return novas_palavras

# Quando há a troca de alguma letra
def troca_letra(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzáàãâéèêíìóòõôúùûç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

# No caso de inverter um caracter. Exemplo: Shireker (Certo: Shrieker)
def inverte_letra(fatias):
    novas_palavras = []
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

# Separando os lados(direito e esquerdo) da palavra, para verficar onde faltou o caracter
def gerador_palavras(palavra):
    fatias = []
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i],palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    palavras_geradas += deletando_caracteres(fatias)
    palavras_geradas += troca_letra(fatias)
    palavras_geradas += inverte_letra(fatias)
    return palavras_geradas

def corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key = probabilidade)
    return palavra_correta

frequencia = nltk.FreqDist(lista_normalizada)
total_palavras = len(lista_normalizada)
frequencia.most_common(10)

def probabilidade(palavra_gerada):
    return frequencia[palavra_gerada] / total_palavras

# Função para avaliar a taxa de acertos do corretor
def cria_dados_teste(nome_arquivo):
    lista_palavras_teste = []
    with open(nome_arquivo, "r", encoding='utf-8') as f:
        for linha in f:
            palavras = linha.split()[:2]  # Pega apenas as duas primeiras palavras
            if len(palavras) == 2:
                correta, errada = palavras
                lista_palavras_teste.append((correta, errada))
            else:
                print(f"A linha '{linha.strip()}' não contém exatamente duas palavras.")
    return lista_palavras_teste

lista_teste = cria_dados_teste("Base/palavras.txt")

palavra = "lóiigica"

# Verificando se há dois erros, exemplo: Liius (Certo = Luís)
def gerador_turbinado(palavras_geradas):
    novas_palavras = []
    for palavra in palavras_geradas:
        novas_palavras += gerador_palavras(palavra)
    return novas_palavras

palavras_g = gerador_turbinado(gerador_palavras(palavra))
vocabulario = set(lista_normalizada)

# Diminuindo o numero de palavras geradas para encontrar a certa no gerador_turbinado
def novo_corretor(palavra):
    palavras_geradas = gerador_palavras(palavra)
    palavras_turbinado = gerador_turbinado(palavras_geradas)
    todas_palavras = set(palavras_geradas + palavras_turbinado)
    candidatos = [palavra]
    for palavra in todas_palavras:
        if palavra in vocabulario:
            candidatos.append(palavra)
    palavra_correta = max(candidatos, key=probabilidade)
    return palavra_correta

novo_corretor(palavra)

#Um corretor que esteja preparado para corrigir tanto palavras com uma letra de distância quanto palavras
#  com duas letras de distância da palavra correta.
def corretor_condicional(palavra):
  palavras_geradas = gerador_palavras(palavra)
  suposta_palavra_correta = max(palavras_geradas,key=probabilidade)
  palavras_conhecidas=[]
  if suposta_palavra_correta in vocabulario:
    palavra_correta = suposta_palavra_correta
  else:
    palavras_geradas_aprimoradas = gerador_turbinado(palavras_geradas)
    soma_palavras = set(palavras_geradas + palavras_geradas_aprimoradas)
    for palavra in soma_palavras:
      if palavra in vocabulario:
          palavras_conhecidas.append(palavra)
    try:
      palavra_correta = max(palavras_conhecidas,key=probabilidade)
    except:
      palavra_correta = palavra
  return palavra_correta

#  Avaliando a taxa de acerto do corretor/novo_corretor
def avaliador(testes, vocabulario):
    numero_palavras = len(testes)
    acertou = 0
    desconhecida = 0
    for correta, errada in testes:
        palavra_corrigida = corretor_condicional(errada)
        desconhecida += (correta not in vocabulario)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou * 100 / numero_palavras, 2)
    taxa_desconhecida = round(desconhecida * 100 / numero_palavras, 2)
    print(f"{taxa_acerto}% de {numero_palavras} palavras, desconhecidas {taxa_desconhecida}%")

avaliador(lista_teste, vocabulario)

palavra = "spenhaki"
print(novo_corretor(palavra))
print(corretor(palavra))
print(corretor_condicional(palavra))

66.14% de 189 palavras, desconhecidas 7.94%
sepanhaki
aspenhaki
sepanhaki
